In [1]:
#### Importing all required libraries
import requests
import pandas as pd
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
ua = UserAgent()
headers = {'user-agent':ua.firefox}

base_url="http://www.pythonhow.com/real-estate/rock-springs-wy/LCWYROCKSPRINGS/t=0&s="
houses_list = []
for page in range(0,30,10):
    #### Requesting the server to get the content of the web page
    page=requests.get(base_url+str(page)+".html", headers=headers, timeout=9)
    
    #### Parsing the html
    soup=BeautifulSoup(page.content,'html.parser')
    
    #### Extracting table data using class name
    houses = soup.find_all(class_="propertyRow")
    for house in houses:
        houses_dict = {}
        
        #### Extracting 'Address' of the house
        houses_dict['Address'] = house.find_all(class_ = 'propAddressCollapse')[0].text
        
        #### Extracting 'Locality' of the house
        houses_dict["Locality"] = house.find_all(class_ = 'propAddressCollapse')[1].text
        
        #### Extracting 'Price' of the house
        houses_dict["Price"] = house.find("h4",{"class","propPrice"}).text.replace("\n","").replace(" ","")
        
        #### Extracting info of the 'Beds' in a house
        try:
            houses_dict["Beds"] = house.find(class_ = 'infoBed').find("b").text
        except:
            pass
        
        #### Extracting 'Area' of the house
        try:
            houses_dict["Area"] = house.find(class_ = 'infoSqFt').find("b").text
        except:
            pass
        
        #### Extracting info of the 'Full Baths' in a house
        try:
            houses_dict["Full Baths"] = house.find(class_ = 'infoValueFullBath').find("b").text
        except:
            pass
        
        #### Extracting 'info of the 'Half Baths' in a house
        try:
            houses_dict["Half Baths"] = house.find(class_ = 'infoValueHalfBath').find("b").text
        except:
            pass
        #### Extracting 'Property Description' of the house
        try:
            houses_dict["PROPERTY DESCRIPTION"] = house.find(class_ = 'propertyDescription').text[25:-1]
        except:
            pass
        
        #### Extracting 'Features' of the house
        for columnGroup in house.find_all(class_ = 'columnGroup'):
            for feature_group, feature_name in zip(columnGroup.find_all(class_ = 'featureGroup'), columnGroup.find_all(class_ = 'featureName')):
                
                #### Extracting 'Lot Size' of the house
                if "Lot Size" in feature_group.text:
                    houses_dict["Lot Size"] = feature_name.text
                
                #### Extracting 'Age' of the house
                if "Age" in feature_group.text:
                    houses_dict["Age"] = feature_name.text
                
                #### Extracting 'Appliances' of the house
                if "Appliances:" in feature_group.text:
                    houses_dict["Appliances"] = feature_name.text
                
                #### Extracting 'Basement' of the house
                if "Basement" in feature_group.text:
                    houses_dict["Basement"] = feature_name.text
        houses_list.append(houses_dict)

#### Converting all the extracted elements into dataframe
houses_df = pd.DataFrame(houses_list) 

#### Printing top 5 rows of the data
houses_df.head()

,Address,Locality,Price,PROPERTY DESCRIPTION,Beds,Full Baths,Age,Appliances,Basement,Lot Size,Area,Half Baths
0,0 Gateway,"Rock Springs, WY 82901","$725,000",This Location could be used for almost anythin...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1003 Winchester Blvd.,"Rock Springs, WY 82901","$452,900",Beautiful new construction located on a corner...,4,4,New Construction,"Dishwasher,",Finished,0.21 Acres,NaN,NaN
2,600 Talladega,"Rock Springs, WY 82901","$396,900",First Built in 2009 and upgraded this year. He...,5,3,6-10 Years Old,"Dishwasher,",NaN,NaN,"3,154",NaN
3,3239 Spearhead Way,"Rock Springs, WY 82901","$389,900",Beautiful Custom built home on large corner lo...,4,3,1-5 Years Old,"Dishwasher,",Full,"Under 1/2 Acre,","3,076",1
4,522 Emerald Street,"Rock Springs, WY 82901","$254,000",Bi-level home in established neighborhood.,3,3,31-40 Years Old,"Dishwasher,",Full,"Under 1/2 Acre,","1,172",NaN


In [2]:
#### Saving the data in to csv file
houses_df.to_csv('houses.csv')